In [1]:
import pandas as pd

In [2]:
hos_refferals = pd.read_csv('../data/hos_referrals.csv')
neo_communites =  pd.read_csv('../data/neo_communities.csv')
nashville_npi = pd.read_csv('../data/nashville_npi.csv')

In [4]:
hos_refferals = hos_refferals.merge(neo_communites, how = 'left', left_on = 'from_npi', right_on = 'npi'
                   ).rename(columns={"communityId": "from_communityId"}
                           ).merge(neo_communites, how = 'left', left_on = 'to_npi', right_on = 'npi'
                                  ).rename(columns={"communityId": "to_communityId"}
                                          ).drop(columns = ['npi_x', 'npi_y'])

In [9]:
hos_refferals = hos_refferals[~hos_refferals['from_classification'].isin(['Radiology', 'Pathology'])]

In [17]:
from_data = hos_refferals[['from_npi', 'from_organization', 'from_classification', 'from_specialization', 'from_communityId']
                         ].drop_duplicates()

In [23]:
to_data = hos_refferals[['from_npi', 'to_organization', 'patient_count', 'to_communityId']
             ].groupby(by = ['from_npi', 'to_organization', 'to_communityId'], dropna = False).sum().reset_index()

In [29]:
patient_totals = hos_refferals[['from_npi', 'patient_count']].groupby(by = ['from_npi'], dropna = False
                                                                     ).sum().rename(columns={"patient_count": "patient_totals"}
                                                                                   ).reset_index()

In [34]:
refferals = to_data.merge(from_data, how = 'left', on = 'from_npi').merge(patient_totals, how = 'left', on = 'from_npi')

In [33]:
cols = [
    'from_npi', 
    'from_organization', 
    'from_classification', 
    'from_specialization', 
    'from_communityId', 
    'to_communityId', 
    'to_organization', 
    'patient_count', 
    'patient_totals']

In [36]:
refferals = refferals[cols]

In [37]:
refferals['patient_percent'] = refferals['patient_count'] / refferals['patient_totals']

In [56]:
vandy_low = refferals[(refferals['to_organization'] == 'VANDERBILT UNIVERSITY MEDICAL CENTER') & 
          (refferals['patient_percent'] < .5) &
          (refferals['patient_totals'] > 1000)].sort_values(by = ['patient_percent']).reset_index().drop(columns = ['index'])

In [61]:
vandy_low = vandy_low[vandy_low['from_npi'].isin(nashville_npi['npi'].to_list())]

In [67]:
vandy_low[~vandy_low['from_classification'].isin(['Durable Medical Equipment & Medical Supplies', 'Ambulance'])
         ].reset_index().drop(columns = ['index'])

,from_npi,from_organization,from_classification,from_specialization,from_communityId,to_communityId,to_organization,patient_count,patient_totals,patient_percent
0,1194758979,MIDDLE TENNESSEE EMERGENCY PHYSICIANS PC,Emergency Medicine,NaN,29,268,VANDERBILT UNIVERSITY MEDICAL CENTER,61,12961,0.004706
1,1679560023,COLUMBIA MEDICAL GROUP - THE FRIST CLINIC INC,Internal Medicine,NaN,195,10,VANDERBILT UNIVERSITY MEDICAL CENTER,94,13052,0.007202
2,1235186800,"PATHGROUP LABS, LLC",Clinical Medical Laboratory,NaN,2210,268,VANDERBILT UNIVERSITY MEDICAL CENTER,120,14225,0.008436
3,1912953381,CARDIOVASCULAR ANESTHESIOLOGISTS PC,Anesthesiology,Critical Care Medicine,1209,10,VANDERBILT UNIVERSITY MEDICAL CENTER,143,13560,0.010546
4,1982013728,MARATHON INPATIENT SERVICES PLLC,Internal Medicine,NaN,195,10,VANDERBILT UNIVERSITY MEDICAL CENTER,63,5796,0.010870
5,1548284060,HERITAGE MEDICAL ASSOCIATES PC,Internal Medicine,NaN,10,10,VANDERBILT UNIVERSITY MEDICAL CENTER,272,19044,0.014283
6,1417983099,THE SURGICAL CLINIC PLLC,Surgery,NaN,10,10,VANDERBILT UNIVERSITY MEDICAL CENTER,199,11262,0.017670
7,1295780476,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,NaN,195,10,VANDERBILT UNIVERSITY MEDICAL CENTER,85,4562,0.018632
8,1104202761,VANDERBILT UNIVERSITY MEDICAL CENTER,Internal Medicine,NaN,268,10,VANDERBILT UNIVERSITY MEDICAL CENTER,1714,91531,0.018726
9,1780778969,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,NaN,10,268,VANDERBILT UNIVERSITY MEDICAL CENTER,93,4695,0.019808
